In [1]:

import pandas
from data_algebra.data_ops import *


d = pandas.DataFrame({
    'c': ['c', 'c', 'b', 'a'],
    'v': [1, 2, 3, 4],
})

d

,c,v
0,c,1
1,c,2
2,b,3
3,a,4


In [2]:
table_name = 'data-algebra-test.test_1.d'

operations = describe_table(d, table_name=table_name) .\
    extend({
        'g': '"prefix_" %+% c'
         }) .\
    project({
        'group_total': 'v.sum()'
        },
        group_by=['g']
        ) .\
    order_rows(['g'])


In [3]:
res_pandas = operations.transform(d)

res_pandas

,g,group_total
0,prefix_a,4
1,prefix_b,3
2,prefix_c,3


In [4]:
import os
from google.cloud import bigquery
import data_algebra.BigQuery

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/johnmount/big_query/big_query_jm.json"
biqquery_handle = data_algebra.BigQuery.BigQueryModel().db_handle(bigquery.Client())

bigquery_sql = biqquery_handle.to_sql(operations, pretty=True)

print(bigquery_sql)

SELECT `g`,
       `group_total`
FROM
  (SELECT `g`,
          SUM(`v`) AS `group_total`
   FROM
     (SELECT ("prefix_"|| `c`) AS `g`,
             `v`
      FROM `data-algebra-test.test_1.d`) `extend_1`
   GROUP BY `g`) `project_2`
ORDER BY `g`


In [5]:
biqquery_handle.insert_table(d, table_name=table_name, allow_overwrite=True)

res_bigquery = biqquery_handle.read_query(operations)

res_bigquery

,g,group_total
0,prefix_a,4
1,prefix_b,3
2,prefix_c,3


In [6]:
assert res_pandas.equals(res_bigquery)

In [7]:
biqquery_handle.close()

In [8]:
import sqlite3
import data_algebra.SQLite

sqlite_handle = data_algebra.SQLite.SQLiteModel().db_handle(sqlite3.connect(":memory:"))

sqlite_sql = sqlite_handle.to_sql(operations, pretty=True)

print(sqlite_sql)

SELECT "g",
       "group_total"
FROM
  (SELECT "g",
          SUM("v") AS "group_total"
   FROM
     (SELECT ('prefix_'|| "c") AS "g",
             "v"
      FROM "data-algebra-test.test_1.d") "extend_1"
   GROUP BY "g") "project_2"
ORDER BY "g"


In [9]:
sqlite_handle.insert_table(d, table_name=table_name, allow_overwrite=True)
res_sqlite = sqlite_handle.read_query(operations)

res_sqlite

,g,group_total
0,prefix_a,4
1,prefix_b,3
2,prefix_c,3


In [10]:
assert res_sqlite.equals(res_bigquery)

In [11]:
operations

TableDescription(
 table_name='data-algebra-test.test_1.d',
 column_names=[
   'c', 'v']) .\
   extend({
    'g': "'prefix_'.concat(c)"}) .\
   project({
    'group_total': 'v.sum()'},
   group_by=['g']) .\
   order_rows(['g'])